# 01 — Full Pipeline Validation

**Goal**: Verify the complete `PyTorch → Quantize → NANO-RUST` pipeline.

**Model**: `Conv2D(1→4, 3×3) → ReLU → Flatten → Dense(144→10)`

**What this notebook demonstrates**:
1. Define a CNN model in PyTorch
2. Quantize weights to i8 using `quantize_weights()`
3. Calibrate requantization using `calibrate_model()`
4. Build the same model in NANO-RUST with i8 weights
5. Compare outputs bit-by-bit

**Prerequisites**: `pip install nano-rust-py numpy torch`

In [ ]:
import sys

import numpy as np
import torch
import torch.nn as nn

from nano_rust_py.utils import quantize_to_i8, quantize_weights, calibrate_model
import nano_rust_py

print('✅ All imports OK')

## Step 1: Define PyTorch Model

A simple CNN for 8×8 single-channel images (like a tiny grayscale sensor grid).

| Layer | Input | Output | Params |
|-------|-------|--------|--------|
| Conv2d(1→4, 3×3) | [1, 8, 8] | [4, 6, 6] | 36 + 4 = 40 |
| ReLU | [4, 6, 6] | [4, 6, 6] | 0 |
| Flatten | [4, 6, 6] | [144] | 0 |
| Linear(144→10) | [144] | [10] | 1440 + 10 = 1450 |
| **Total** | | | **1,490** |

In [ ]:
torch.manual_seed(42)
model = nn.Sequential(
    nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=0, bias=True),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(4 * 6 * 6, 10, bias=True),
)
model.eval()

print(model)
print(f'Total parameters: {sum(p.numel() for p in model.parameters()):,}')
print(f'Float32 size: {sum(p.numel() for p in model.parameters()) * 4:,} bytes')
print(f'Int8 size:    {sum(p.numel() for p in model.parameters()):,} bytes (4× smaller)')

## Step 2: Quantize Weights & Calibrate

**Quantization**: Convert float32 weights to i8 using symmetric linear scaling.
- Each layer's max absolute weight maps to ±127
- Scale factor = max_abs / 127

**Calibration**: Run a sample input through the float model to compute:
- `requant_m`: Fixed-point multiplier for `(acc × M) >> shift`
- `requant_shift`: Bit-shift for the above formula
- `bias_corrected`: Bias pre-scaled to match output quantization scale

In [ ]:
# Quantize all model weights to i8
q_weights = quantize_weights(model)
for name, info in q_weights.items():
    print(f'Layer {name} ({info["type"]}): '
          f'{info["weights"].shape} → {info["weights"].size} bytes, '
          f'scale={info["weight_scale"]:.6f}')

# Create a random test input
torch.manual_seed(42)
input_tensor = torch.randn(1, 1, 8, 8)  # batch=1, channels=1, 8×8
q_input, input_scale = quantize_to_i8(input_tensor.numpy().flatten())
print(f'\nInput scale: {input_scale:.6f}')
print(f'Input range: [{q_input.min()}, {q_input.max()}]')

# Calibrate requantization parameters
requant = calibrate_model(model, input_tensor, q_weights, input_scale)
print('\n=== Calibrated Requant Params ===')
for name, (m, s, bc) in requant.items():
    print(f'  Layer {name}: M={m}, shift={s}, bias_corrected={bc}')

## Step 3: Build NANO-RUST Model & Compare

Now we build the exact same model architecture using `nano_rust_py`:
- Use i8 weights from `quantize_weights()`
- Use calibrated `requant_m` and `requant_shift` from `calibrate_model()`
- Use corrected bias (already in output scale)

Then run the same input through both and compare the output.

In [ ]:
# --- PyTorch reference output ---
with torch.no_grad():
    pytorch_float_output = model(input_tensor).numpy().flatten()
q_pytorch, _ = quantize_to_i8(pytorch_float_output)

# --- NANO-RUST model ---
nano = nano_rust_py.PySequentialModel(input_shape=[1, 8, 8], arena_size=32768)

# Add Conv2D with calibrated requant
conv_m, conv_s, conv_bias = requant['0']
nano.add_conv2d_with_requant(
    q_weights['0']['weights'].flatten().tolist(),
    conv_bias,
    1, 4, 3, 3, 1, 0,  # in_ch, out_ch, kh, kw, stride, padding
    conv_m, conv_s
)
nano.add_relu()
nano.add_flatten()

# Add Dense with calibrated requant
dense_m, dense_s, dense_bias = requant['3']
nano.add_dense_with_requant(
    q_weights['3']['weights'].flatten().tolist(),
    dense_bias,
    dense_m, dense_s
)

# Run inference
nano_output = nano.forward(q_input.tolist())
nano_arr = np.array(nano_output, dtype=np.int8)

# --- Compare ---
diff = np.abs(q_pytorch.astype(np.int32) - nano_arr.astype(np.int32))
TOLERANCE = 20

print('=' * 50)
print('       PIPELINE VALIDATION RESULTS')
print('=' * 50)
print(f'PyTorch (i8): {q_pytorch.tolist()}')
print(f'NANO-RUST:    {nano_arr.tolist()}')
print(f'Max diff:     {int(np.max(diff))}')
print(f'Mean diff:    {float(np.mean(diff)):.2f}')
print(f'PyTorch class:  {int(np.argmax(q_pytorch))}')
print(f'NANO-RUST class:{int(np.argmax(nano_arr))}')
print('-' * 50)
ok = int(np.max(diff)) <= TOLERANCE
print(f'{"✅ Numerical: PASS" if ok else "❌ Numerical: FAIL"} (tolerance={TOLERANCE})')
match = int(np.argmax(q_pytorch)) == int(np.argmax(nano_arr))
print(f'{"✅ Classification: AGREE" if match else "⚠️ Classification: DISAGREE"}')
print('=' * 50)
print('🎉 PASSED!' if ok and match else '⚠️ Check results above')